In [1]:
# Import libraries here that you need for different processing steps
import nltk
import csv
import spacy
import pandas as pd

## Loading the dataset


In [2]:
# Read the csv file into a dataframe

data_file = "./Dataset/covid.csv"

data_df = pd.read_csv(data_file)
print ("Training set: ", len(data_df))

display(data_df)

Training set:  41159


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
...,...,...,...,...,...,...
41154,44951,89903,"Wellington City, New Zealand",14-04-2020,Airline pilots offering to stock supermarket s...,Neutral
41155,44952,89904,NaN,14-04-2020,Response to complaint not provided citing COVI...,Extremely Negative
41156,44953,89905,NaN,14-04-2020,You know itÂs getting tough when @KameronWild...,Positive
41157,44954,89906,NaN,14-04-2020,Is it wrong that the smell of hand sanitizer i...,Neutral


## Basic Data Cleaning and Transformations

## Counting Sentiment Labels

In [3]:
# Run the cell to see the distribution of the sentiment label in the dataset

print(data_df.Sentiment.value_counts())

Positive              11422
Negative               9917
Neutral                7711
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64


## Dropping Columns
Identify the column that you think is less relevant for text mining or other NLP tasks

In [4]:
data_df.drop(['ScreenName','TweetAt'],axis=1,inplace=True)
print(data_df.shape)
data_df.head(10)

(41159, 4)


,UserName,Location,OriginalTweet,Sentiment
0,3799,London,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,UK,advice Talk to your neighbours family to excha...,Positive
2,3801,Vagabonds,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,NaN,My food stock is not the only one which is emp...,Positive
4,3803,NaN,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,"ÃT: 36.319708,-82.363649",As news of the regionÂs first confirmed COVID...,Positive
6,3805,"35.926541,-78.753267",Cashier at grocery store was sharing his insig...,Positive
7,3806,Austria,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,"Atlanta, GA USA",Due to COVID-19 our retail store and classroom...,Positive
9,3808,"BHAVNAGAR,GUJRAT","For corona prevention,we should stop to buy th...",Negative


## Handling NULL values
Handle null values in a column by specifying the alternate value

In [14]:
filled_col_df = data_df['Sentiment'].fillna("NA")
filled_col_df


0                   Neutral
1                  Positive
2                  Positive
3                  Positive
4        Extremely Negative
                ...        
41154               Neutral
41155    Extremely Negative
41156              Positive
41157               Neutral
41158              Negative
Name: Sentiment, Length: 41159, dtype: object

In [15]:
# run this cell to fill all null values in all columns of the dataframe with the desired value. 
# Trying doing it for a single column yourself.

data_df = data_df.fillna("NA")
data_df.head()

,UserName,Location,OriginalTweet,Sentiment
0,3799,London,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,UK,advice Talk to your neighbours family to excha...,Positive
2,3801,Vagabonds,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,NA,My food stock is not the only one which is emp...,Positive
4,3803,NA,"Me, ready to go at supermarket during the #COV...",Extremely Negative


## Handling Exceptions
Use try-except-pass

In [17]:
# Incorporate the statements within a try-except block where you suspect there might be errors.
# In the except block, handle the exception according to the requirement.

try:
    data_df=data_df.fillna("NA")
except:
    pass

## Lowercase conversion

In [22]:
# Example showing lower case conversion of Tweet for an instance

print(data_df.OriginalTweet.tolist()[100])
print("\n")
print(data_df.OriginalTweet.tolist()[100].lower())

## Handling Special Characters or links

This should be an interesting step. You can remove special characters or links or anything that does not have value for the sentiment

In [23]:
# Example showing removal of special chars from the Tweet for an instance

import re

print(data_df.OriginalTweet.tolist()[100])
print("\n")
print(re.sub('[^A-Za-z0-9]+', ' ', data_df.OriginalTweet.tolist()[100]))

## TEXT PRE-PROCESSING

The following are the techniques to tranform the data into a cleaner data. Try out the what all techniques you would apply to your textual data to get the best quality dataset for a model.
You can use either/all/addition to these steps mentioned below in any order that you find appropriate.

- Tokenization
- Sentence Segmentation

- Stemming
- Lemmatization
- PartOfSpeech Tagging
- Others


In [24]:
# Run this cell to see an example of text pre-processing including a few of these techniques. 
# This image is a hint to the pre-processing steps but may or may not be the best. 
# It shows how the input text changes with each processing step
# Find out different Tokenizers, Stemmers, Lemmatizers, etc and try to use the best one for your task! 

from PIL import Image
pil_im = Image.open('./Dataset/Text_preprocessing_example.png')
display(pil_im)

## Tokenization using nltk library
 Links to a few word and sentence tokenizers- https://www.nltk.org/howto/tokenize.html

In [25]:
# Example showing tokenization of the Tweets into words for an instance

from nltk.tokenize import word_tokenize  

print(data_df.OriginalTweet.tolist()[100])
print("\n")
print(word_tokenize(data_df.OriginalTweet.tolist()[100]))

## Stemming
Links to a few nltk Stemmers- https://www.nltk.org/howto/stem.html

In [26]:
# Example showing stemming of words

from nltk.stem import PorterStemmer 
ps = PorterStemmer() 

print(data_df.OriginalTweet.tolist()[51])
print("\n")

tweet = data_df.OriginalTweet.tolist()[51].split()

for word in tweet:
    print(word," ",ps.stem(word) ," ")